# Training
Adjust as needed for project versions and testing

## Imports

In [ ]:
import numpy as np
import os
import librosa as lb
import soundfile as sf

import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.backend import clear_session
from sklearn.model_selection import train_test_split

import pickle
import json
import time
import sys

## Parameters

In [ ]:
sampleRate = 44100
channels = 1


learningRate = [0.01, 0.001, 0.0001]

#samples relative to sampleRate
#44 = 1ms at 44.1K Hz
context = [44, 66, 88]

#recorded gain values
gain = ["Low", "Mid", "High"]

lstmUnits = [64, 96]

#SD-1
effect = ""
#DS340
projVer = ""

patience = 15
batch = 64
epoch = 100
shuf = True

state = 25
np.random.seed(state)

## Functions

In [ ]:
def esrLoss(true, pred):
    return tf.reduce_sum(tf.pow(true - pred,2), axis = 0) / (tf.reduce_sum(tf.pow(true, 2), axis = 0) + 1e-50)

def createModel(units, rate):
    #mess with input shapes later
    inputLayer = Input(shape = (None,1))
    lstm = LSTM(units)(inputLayer)
    fullyConnected = Dense(1)(lstm)
    model = Model(inputs = inputLayer, outputs = fullyConnected)
    model.compile(optimizer = Adam(learning_rate = rate), loss = esrLoss, metrics = ["mae", "mse"])
    return model

def predictionPrep(audio, context):
    chunks = []
    for i in range(len(audio) - context):
        chunks.append(audio[i:i+context])
    return np.array(chunks)

def evaluateNew(model, xTest, yTest, context):
    yPred = model.predict(predictionPrep(xTest, context))
    return esrLoss(yTest[context:], yPred.squeeze()), yPred

## Execute

In [ ]:
#reference: https://rcs.bu.edu/examples/machine_learning/
gpuIds = os.environ["CUDA_VISIBLE_DEVICES"].split(",")
strategy = tf.distribute.MirroredStrategy()


modelNum = 0

for lvl in gain:
    for size in context:
    

        with open(f"../TrainValPickles/{effect}/{projVer}/{lvl}_{size}_x.pkl", "rb") as file1:
            x = pickle.load(file1)

        with open(f"../TrainValPickles/{effect}/{projVer}/{lvl}_{size}_y.pkl", "rb") as file2:
            y = pickle.load(file2) 
        
      
        #these should be adjusted with the size of the training data
        testSize = 0.0
        if size == 44:
            testSize = 0.6
        if size == 66:
            testSize = 0.4
        if size == 88:
            testSize = 0.2

        #add a way to train on the WHOLE dataset
        #use external validation data
        xTrain, xVal, yTrain, yVal = train_test_split(x, y, test_size = testSize, random_state = state)
    
        #evaluate on shadowLove
        evalDry, _ = lb.load(f"../Data/{effect}/{projVer}/Test/{lvl}/Dry/d_shadowlove.wav", sr=sampleRate, mono=True)
        evalWet, _ = lb.load(f"../Data/{effect}/{projVer}/Test/{lvl}/Wet/w_shadowlove.wav", sr=sampleRate, mono=True)
        
        
        for units in lstmUnits:
            for rate in learningRate:
                
                clear_session()
                directory = f"../Models/{effect}/{projVer}/Model_{modelNum}"
                model = createModel(units, rate)

                os.makedirs(directory, exist_ok=True)
                #optional for monitoring model
                """
                with open(f"{directory}/modelSummary_{modelNum}.txt", "w") as file1:
                    sys.stdout = file1
                    model.summary()  
                    sys.stdout = sys.__stdout__
                """
                earlyStop = EarlyStopping(monitor = 'val_loss', patience = patience, restore_best_weights = True)
                start = time.time()
                #use test to run Validation, will use my data for testing
                history = model.fit(xTrain, yTrain, validation_data = (xVal, yVal), batch_size = batch, epochs = epoch, callbacks = [earlyStop], shuffle = shuf)
                end = time.time()
                trainTime = end - start

                testing, output = evaluateNew(model, evalDry, evalWet, size)

                parameters = {
                    "sampleRate" : int(sampleRate),
                    "contextSize" : int(size),
                    "trainNvalSize" : xTrain.shape[0],
                    "lstmUnits" : int(units),
                    "learningRate" : float(rate),
                    "trainEpochs" : int(epoch),
                    "epochs" : len(history.history["loss"]),
                    "patience" : int(patience),
                    "batchSize" : int(batch),
                    "shuffle" : shuf,
                    "randomState" : int(state),
                    "trainTime": trainTime,
                    "gain" : str(lvl),
                    "testESR" : float(testing.numpy())
                }


                modelName = f"{lvl}_{units}_{rate}_{size}_{modelNum}"
                #or use PCM_16 or something
                sf.write(f"{directory}/{modelName}.wav", output, sampleRate, subtype = "PCM_24")
                
                model.save(f"{directory}/model_{modelName}.h5")

                with open(f"{directory}/history_{modelName}.json", "w") as file3:
                    json.dump(history.history, file3)

                with open(f"{directory}/parameters_{modelName}.json", "w") as file4:
                    json.dump(parameters, file4)

                #print("finished model " + modelName)
                modelNum += 1